En este notebook **proceso** las columnas

# Libraries

In [41]:
# Data management
try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

try:
    import numpy as np
except:
    %pip install numpy
    import numpy as np

import os

In [42]:
# Plots
try:
    import matplotlib.pyplot as plt
except:
    %pip install matplotlib
    import matplotlib.pyplot as plt

try:
    import seaborn as sns
except:
    %pip install seaborn
    import seaborn as sns
sns.set_style("white")

In [43]:
# Warnings
try:
    import warnings as ws
except:
    %pip install warnings
    import warnings as ws
ws.filterwarnings ("ignore")

In [44]:
# ML
try:
    import xgboost
except:
    %pip install xgboost
    import xgboost

from sklearn import datasets, ensemble, model_selection

In [45]:
# XAI
# SHAP
try:
  import shap
except:
  %pip install shap
  import shap

# Data

In [46]:
current_path = os.getcwd()
src_file_name = "compas.csv"
src_path = os.path.join(current_path, "data_src")
src_file = os.path.join(src_path, src_file_name)

In [47]:
# Load
df = pd.read_csv(src_file)

In [48]:
len(df.columns)

53

In [49]:
df['decile_score']

0       1
1       3
2       4
3       8
4       1
       ..
7209    7
7210    3
7211    1
7212    2
7213    4
Name: decile_score, Length: 7214, dtype: int64

In [50]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [51]:
df_backup = df.copy()

# Processing

In [52]:
# list_scores_text_equals = (df["v_score_text"] == df["score_text"])
# list_scores_text_equals.sum()/len(list_scores_text_equals)*100
# list_decil_scores_equals = (df["v_decile_score"] == df["decile_score"])
# list_decil_scores_equals.sum()/len(list_decil_scores_equals)*100

In [53]:
# drop columns
list_drop = ["id", "name", "dob", "days_b_screening_arrest", "c_case_number", "c_offense_date", "c_arrest_date", "r_case_number", "r_charge_degree", "c_charge_desc",
      "r_days_from_arrest", "r_offense_date", "r_charge_desc","violent_recid", "vr_case_number", "vr_charge_degree", "vr_offense_date", "vr_charge_desc", "first", "last",
      "type_of_assessment", "start", "end", "event", "v_type_of_assessment", "v_screening_date", "compas_screening_date", "screening_date", "age_cat", "priors_count.1"]
list_drop.append("v_score_text")
list_drop.append("score_text")
df = df.drop(list_drop, axis=1)

In [54]:
# variables salida
# 1. v_decile_sore
# y1 = df["v_decile_score"]
# df = df.drop(["v_decile_score"], axis=1)
# df = df.drop(["decile_score"], axis=1)
# df = df.drop(["decile_score.1"], axis=1)
# 2. v_decile_text
# y2 = df["v_score_text"]
# df = df.drop(["v_score_text"], axis=1)
# df = df.drop(["score_text"], axis=1)
# 3. two_year_recid
# y3 = df["two_year_recid"]
# df = df.drop(["two_year_recid"], axis=1)
# 4. is_recid
# y3 = df["is_recid"]
# df = df.drop(["is_recid"], axis=1)

In [55]:
def leap_year(year):
    return year%4 == 0

dic_days_mont = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

def top_days(month_tmp, year_tmp):
    top = dic_days_mont[month_tmp]
    if month_tmp == 2 and leap_year(year_tmp):
        top += 1
    return top

def get_days_distance(year_in, month_in, day_in, year_out, month_out, day_out, it):
    # if it == 25:
    #     print(f"Date in: day {day_in}, month {month_in} and year {year_in}")
    #     print(f"Date out: day {day_out}, month {month_out} and year {year_out}")
    #     print(f"it={it}")
    days = 0
    day_tmp = day_in
    month_tmp = month_in
    year_tmp = year_in
    while day_tmp!=day_out or month_tmp!=month_out or year_tmp!=year_out:
        # if it == 25: print(f"Date tmp: day {day_tmp}, month {month_tmp} and year {year_tmp}")
        days += 1
        if day_tmp < top_days(month_tmp, year_tmp):
            day_tmp += 1
            # if it == 25: print(f"Next day ({day_tmp}) for the iteration {it}")
        else:
            day_tmp = 1
            if month_tmp == 12:
                month_tmp = 1
                year_tmp += 1
                # if it == 25: print(f"Next year ({year_tmp}) for the iteration {it}")
            else:
                month_tmp +=1
                # if it == 25: print(f"Next month ({month_tmp}) for the iteration {it}")
    # We count the last day in jail
    days += 1
    return days

In [56]:
def count_days(date_1, date_2, it):
    # print(f"it={it}")
    days = 0
    year_in = int(date_1[0])
    month_in = int(date_1[1])
    day_in = int(date_1[2])
    year_out = int(date_2[0])
    month_out = int(date_2[1])
    day_out = int(date_2[2])
    # print(f"Date in: day {day_in}, month {month_in} and year {year_in}")
    # print(f"Date out: day {day_out}, month {month_out} and year {year_out}")
    days += get_days_distance(year_in, month_in, day_in, year_out, month_out, day_out, it)    
    return days

In [57]:
df["days_custody"] = 0
df["days_c_jail"] = 0
df["days_r_jail"] = 0
for i in range(len(df)):
    # print(f"it {i+1}/{len(df)}")
    if pd.notna(df.loc[i,"in_custody"]) and pd.notna(df.loc[i,"out_custody"]):
        df.loc[i, "days_custody"] = count_days(df.loc[i,"in_custody"].split("-"), df.loc[i,"out_custody"].split("-"), i+1)

    if pd.notna(df.loc[i,"c_jail_in"]) and pd.notna(df.loc[i,"c_jail_out"]):
        df.loc[i, "days_c_jail"] = count_days(df.loc[i,"c_jail_in"].split(" ")[0].split("-"), df.loc[i,"c_jail_out"].split(" ")[0].split("-"), i+1)

    if pd.notna(df.loc[i,"r_jail_in"]) and pd.notna(df.loc[i,"r_jail_out"]):
        df.loc[i, "days_r_jail"] = count_days(df.loc[i,"r_jail_in"].split(" ")[0].split("-"), df.loc[i,"r_jail_out"].split(" ")[0].split("-"), i+1)

df = df.drop(['in_custody', 'out_custody'], axis=1)
df = df.drop(['c_jail_in', 'c_jail_out'], axis=1)
df = df.drop(['r_jail_in', 'r_jail_out'], axis=1)
        

In [58]:
# # drop na
print(len(df))
# print(df.isnull().sum())
df.dropna(inplace=True)
# # Reset index
df = df.reset_index()
df = df.drop(['index'], axis=1)
# print(df.isnull().sum())
print(len(df))

7214
7192


In [59]:
df.loc[df["race"] == "Other", "race"] = 0
df.loc[df["race"] == "African-American", "race"] = 1
df.loc[df["race"] == "Caucasian", "race"] = 2
df.loc[df["race"] == "Hispanic", "race"] = 3
df.loc[df["race"] == "Native American", "race"] = 4
df.loc[df["race"] == "Asian", "race"] = 5

df.loc[df["sex"] == "Male", "sex"] = 0
df.loc[df["sex"] == "Female", "sex"] = 1

df.loc[df["c_charge_degree"] == "M", "c_charge_degree"] = 0
df.loc[df["c_charge_degree"] == "F", "c_charge_degree"] = 1

In [60]:
df.head()

,sex,age,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,c_days_from_compas,c_charge_degree,is_recid,is_violent_recid,decile_score.1,v_decile_score,two_year_recid,days_custody,days_c_jail,days_r_jail
0,0,69,0,0,1,0,0,0,1.0,1,0,0,1,1,0,8,2,0
1,0,34,1,0,3,0,0,0,1.0,1,1,1,3,1,1,11,11,0
2,0,24,1,0,4,0,1,4,1.0,1,1,0,4,3,1,1,2,1
3,0,23,1,0,8,1,0,1,1.0,1,0,0,8,6,0,0,0,0
4,0,43,0,0,1,0,0,2,76.0,1,0,0,1,1,0,0,0,0


In [61]:
rs = np.random.RandomState(0)
corr = df.corr()
corr.style.background_gradient(cmap="coolwarm").format("{:.2f}") 

,sex,age,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,c_days_from_compas,c_charge_degree,is_recid,is_violent_recid,decile_score.1,v_decile_score,two_year_recid,days_custody,days_c_jail,days_r_jail
sex,1.00,0.00,0.02,-0.06,-0.06,-0.05,-0.06,-0.12,0.00,-0.05,-0.10,-0.06,-0.06,-0.12,-0.09,-0.04,-0.06,-0.06
age,0.00,1.00,0.12,-0.07,-0.39,-0.12,-0.16,0.14,0.09,-0.09,-0.20,-0.10,-0.39,-0.58,-0.19,-0.01,0.01,-0.05
race,0.02,0.12,1.00,-0.05,-0.17,-0.07,-0.03,-0.12,-0.02,-0.07,-0.09,-0.04,-0.17,-0.18,-0.08,-0.04,-0.02,-0.05
juv_fel_count,-0.06,-0.07,-0.05,1.00,0.18,0.09,0.05,0.18,0.00,0.05,0.09,0.06,0.18,0.17,0.09,0.06,0.06,0.11
decile_score,-0.06,-0.39,-0.17,0.18,1.00,0.22,0.18,0.44,0.03,0.18,0.35,0.17,1.00,0.78,0.35,0.16,0.21,0.19
juv_misd_count,-0.05,-0.12,-0.07,0.09,0.22,1.00,0.27,0.24,0.01,0.03,0.11,0.09,0.22,0.21,0.11,0.03,0.04,0.03
juv_other_count,-0.06,-0.16,-0.03,0.05,0.18,0.27,1.00,0.10,0.00,0.03,0.11,0.05,0.18,0.20,0.11,0.10,0.04,0.02
priors_count,-0.12,0.14,-0.12,0.18,0.44,0.24,0.10,1.00,0.01,0.14,0.28,0.09,0.44,0.21,0.28,0.13,0.20,0.14
c_days_from_compas,0.00,0.09,-0.02,0.00,0.03,0.01,0.00,0.01,1.00,-0.05,-0.07,-0.01,0.03,-0.01,-0.07,0.05,0.02,-0.01
c_charge_degree,-0.05,-0.09,-0.07,0.05,0.18,0.03,0.03,0.14,-0.05,1.00,0.10,-0.01,0.18,0.14,0.11,0.08,0.12,0.09


## Export csv

In [65]:
dst_file_name = "processed_compas.csv"
dst_path = os.path.join(current_path, "data_dst")
dst_file = os.path.join(dst_path, dst_file_name)

In [66]:
df.to_csv(dst_file, index = False, header=True)